# Sales Channel Prediction Case Study

## Prelude:
        In any firm sales form the bread and butter to an organization. Sales is one of the factors and quite crucial in         determining the survival and prosperity of an organization. So continuing our discussion, this project is regarding the         marketing mix an organization uses to boost sales. For this task, one analyst has to determine the perfect ratio of     media channels engaged by the firm to maximise the sales by allocating optimum budget to each medium of communication.

### About the Independent Variables:
 We are given three continuous variables which are as follows:
 
 1. TV: This variable talks about expenditure in Television advertisements
 2. Radio: This variable talks about expenditure in Radio advertisements
 3. Newspaper: This variable talks about expenditure in Newspaper advertisements

### About the Dependent Variable:
 
 Sales: This continuous variable talks about the sales generated by employing the above independent variables.

## Problem Statement:
    Build a best-fit prediction model for allocation of budget in media channels to gain optimum sales 

In [ ]:
# Importing necessary packages

# Importing fundamental packages
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell        ## To display multiple outputs
InteractiveShell.ast_node_interactivity = "all"
import pyforest            

## For visualization
import matplotlib.pyplot as plt                                         
%matplotlib inline

import seaborn as sns

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

## Data Pre-Processing Packages
from scipy.stats import chi2_contingency
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
%pylab inline
from pylab import *
import random as pyrandom
from scipy.spatial.distance import cdist
matplotlib.rc("image",cmap="gray")
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE
from sklearn.preprocessing import scale



## To create copy of data
import copy

## Pipeline Packages

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# For Building Model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor




## Evaluation Metrics Packages
import statsmodels.api as sm
from scipy.stats import f
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Saving the model
import pickle


In [ ]:
def read(link):
    global data

    data=pd.read_csv(link)
    
    data=pd.DataFrame(data)
    print(data)

In [ ]:
read(link="Advertising.csv")

In [ ]:
def eda(mydata):                                    # Defining function
    
    pd.set_option("display.max_rows", None)         # to display all rows
    pd.set_option("display.max_columns", None)       #to display all columns
    
    print(mydata.head())                              # to display first 10 records
    print("\n")                               
    print(mydata.tail())                              # to display last 10 records
    print("\n")
    print("\n")
    
    print(mydata.info())                               # to understand attributes of the data
    
    print(mydata.describe())                          # to get descriptive statistics
    print("\n")
    print("\n")
    print("Skewness for the data","\n",mydata.skew())       # to get skewness of the data, skewness=0 for normal distribution
    print("\n")
    print("Kurosis for the data","\n",mydata.kurtosis() )            # to get kutosis, kurtosis <=3 for normal distribution
    print("\n")
    print("\n")
    
    sns.pairplot(mydata, kind='scatter', diag_kind='kde')                       # to represent data graphically
    print("\n")
    print("\n")    
    
    plt.figure(figsize=(10,10))                      # plotting heat map to check correlation
    sns.heatmap(mydata.corr(method = "pearson"), annot = True)
    print("\n")    

In [ ]:
eda(mydata=data)

In [ ]:
def outlier(mydata):                        # Outlier Plotting
    for i in mydata.columns:
        fig = px.box(mydata, y= i, width=600, height=400, title=i, template="plotly_dark")
        fig.show()

In [ ]:
outlier(mydata=data)

In [ ]:
## Histogram for all the variables using plotly_express by 'quality'

for i in data.columns:
    fig = px.histogram(data, x= i, histfunc = "count", color = "sales", 
                       width=1000, height=800, title = "Histogram for " + i, 
                       template="plotly_dark", cumulative=True)

    fig.show() 

In [ ]:
# Plotting qq plot

for i in data.columns:
    fig = sm.qqplot(data[i])

    fig.show() 
    

In [ ]:
# Scatterplot for TV and Sales

sns.scatterplot(x="TV",y="sales",data=data)

In [ ]:
# Scatterplot for Radio and Sales

sns.scatterplot(x="radio",y="sales",data=data)

In [ ]:
# Scatterplot for Newspaper and Sales
sns.scatterplot(x="newspaper",y="sales",data=data)

In [ ]:
data_features= pd.DataFrame(data.drop(["sales"], axis=1))

In [ ]:
def calc_vif(data):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = data.columns
    vif["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]

    return(vif)

In [ ]:
calc_vif(data=data.drop(["sales"],axis=1))

## Findings:
1. There are no missing data in the dataset

2. There are 200 rows and 5 columns

3. The data follows a normal distribution

4. The "unnamed:0" column happens to be an identifier as it has only serial number as values and has neglible correlation          suggesting no significance to our target variable. Therefore, it can be dropped from our dataset.

5.  Average expenditure on TV ads is 147.04 and average sales generated is 12089.37

6.  Average expenditure on Radio ads is 23.26 and average sales generated is 3021.92

7.  Average expenditure on Newspaper ads is 30.55 and average sales generated is 3874.63

8. Average sales generated because of media channels is 14.02

9. Newspaper has outliers

10. There is no issue of multi-collinearity in the data

11. Cumulative effect of spending on media results in increasing sales


## Cleaning the dataset

In [ ]:
# Making a copy of dataset for cleaning and model building purpose

data2= copy.deepcopy(data)

In [ ]:
data2= data.drop(["Unnamed: 0"], axis=1)

In [ ]:
# Treating outliers
q1 = data2.quantile(.25)    # first quartile
q3 = data2.quantile(.75)    # third quartile
iqr = q3 - q1

for i in data2.drop(['sales'],axis=1).columns:
    data2['newspaper'] = np.where(data2[i] > (data2[i].quantile(0.75) + (data2[i].quantile(0.75) - data2[i].quantile(0.25))*1.5),
                           (data2[i].quantile(0.75) + (data2[i].quantile(0.75) - data2[i].quantile(0.25))*1.5),
                          np.where(data2[i] < (data2[i].quantile(0.25) - (data2[i].quantile(0.75) - data2[i].quantile(0.25))*1.5),
                           (data2[i].quantile(0.25) - (data2[i].quantile(0.75) - data2[i].quantile(0.25))*1.5),data2[i]))
    



In [ ]:
outlier(mydata=data2)

In [ ]:
data_features2=data2.drop(["sales"], axis=1)

In [ ]:
# DB Scan

# Data creation
centers = [[1, 1], [-1, -1], [1, -1]]
data_features, labels_true = make_blobs(n_samples=50, centers=centers, cluster_std=0.4,
                            random_state=1)  # generate sample blobs
X = StandardScaler().fit_transform(data_features2)
X[:5]

# DBSCAN
db = DBSCAN(eps=0.5, min_samples=3).fit(X)

labels = db.labels_  # similar to the model.fit() method, it gives the labels of the clustered data

print (labels)

# Setting Length of labels
len(set(labels))

1 if -1 in labels else 0

# Number of clusters in labels, ignoring noise if present.

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) # the label -1 is considered as noise by the DBSCAN algorithm
n_noise_ = list(labels).count(-1)  # calculating the number of noises (-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Below code is for showcasing in diagram. Nothing related to model building

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)  # creating an array of true and false as the same size as db.labels

core_samples_mask[db.core_sample_indices_] = True  # setting the indices of the core regions to True
# Black  is used for noise.
unique_labels = set(labels)  # identifying all the unique labels/clusters

colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]  # creating the list of colours, generating the colourmap



for k, col in zip(unique_labels, colors):
    
    
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)   # assigning class members for each class
    

    xy = X[class_member_mask & core_samples_mask] # creating the list of points for each class
    
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)
    

    xy = X[class_member_mask & ~core_samples_mask] # creating the list of noise points
    
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='b', markersize=14)
    
    

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

# Finding counts in each cluster
np.unique(labels, return_counts=True)

# Selecting only noises
noises = X[labels == 4]

print (noises)




There are no outliers

In [ ]:
data_features2= pd.DataFrame(data2.drop(["sales"], axis=1))
data_label=pd.DataFrame(data2["sales"])
print(data_label)

In [ ]:
# Removing skewness using Yeo-Johnson
# Removing data skewness
pt = PowerTransformer(method='yeo-johnson',standardize='True')
    
data_noskew= pt.fit_transform(data_features2.values)
data_noskewtab= pd.DataFrame(data_noskew)
data_noskewtab.columns = ['TV', 'radio', 'newspaper']

print(data_noskewtab)

In [ ]:
data_clean=pd.DataFrame(data_noskewtab)
print(data_clean)

In [ ]:
data_final=pd.DataFrame()
data_final=pd.concat([data_clean,data_label], axis=1)
print(data_final)

## Train Test Split

In [ ]:
def split (data,target):
    data_reset_index = data.reset_index(drop=True)
# Data split
    global x
    global y
    global x_train
    global y_train
    global x_test
    global y_test
# Segregate Feature & Target Variables
    x = data_reset_index.drop(target, axis=1)
    y = data_reset_index[target]
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3, random_state=3)
    
    print(x_train.info())
    ("\n")
    print(x_test.info())
    ("\n")
    print(y_train.shape)
    ("\n")
    print(y_test.shape)
    ("\n")

In [ ]:
split(data=data_final,
      target="sales")

## Scaling the data

In [ ]:
scale_1= StandardScaler()
scale_2= MinMaxScaler()
scale_3= RobustScaler()
model= LinearRegression()

In [ ]:
pipe1 = Pipeline([
    ('Scale',scale_1),
    ('model',model),
])    
pipe2 = Pipeline([
    ('Scale',scale_2),
    ('model',model),
])
pipe3 = Pipeline([
    ('Scale',scale_3),
    ('model',model),  
    
])

In [ ]:
# Create Function Name 
def pre_process(data, pipe):

# Pipe.fit, pipe.predict and accuracy
    
    pipe.fit(x_train,y_train)
    
    y_pred = pipe.predict(x_test)
    
    score = np.sqrt(metrics.mean_squared_error(y_pred, y_test))
    
    return score

In [ ]:
pre_process(data= data2,
           pipe= pipe1)

In [ ]:
pre_process(data= data2,
           pipe= pipe2)

In [ ]:
pre_process(data= data2,
           pipe= pipe3)

Lets go ahead with Robust scaler as its score is maximum

In [ ]:
scalar= RobustScaler()
scalar.fit(x_train)
x_trainsc =  scalar.transform(x_train)
x_testsc  =  scalar.transform(x_test)


## Model Building

Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(x_trainsc, y_train)


model = sm.OLS(y_train, x_trainsc).fit()


y_pred = model.predict(x_testsc)

print("Linear Regression Train Score: ", lr.score(x_trainsc , y_train))

print("Linear Regression Test Score: ", lr.score(x_testsc , y_test))

print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred),2))  
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),2))
print(model.summary())

In [ ]:
rfe = RFE(lr, n_features_to_select=6)             
rfe = rfe.fit(x_trainsc, y_train)

r2 = sklearn.metrics.r2_score(y_test, y_pred)

scores = cross_val_score(lr, x_trainsc, y_train, scoring='r2', cv=5)
scores   

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lr, x_trainsc, y_train, scoring='r2', cv=folds)
scores  

scores = cross_val_score(lr, x_trainsc, y_train, scoring='neg_mean_squared_error', cv=5)
scores

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, 14))}]


# step-3: perform grid search
# 3.1 specify model
lm = LinearRegression()
lm.fit(x_trainsc, y_train)
rfe = RFE(lm)             

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
grid_result= model_cv.fit(x_trainsc, y_train)   
print('Best Score: ', grid_result.best_score_)

print('Best Params: ', grid_result.best_params_)

In [ ]:
# final model
n_features_optimal = 3
lm = LinearRegression()
lm.fit(x_trainsc, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(x_trainsc, y_train)

# predict prices of X_test
y_pred = lm.predict(x_testsc)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

print("Linear Regression Train Score: ", lm.score(x_trainsc , y_train))

print("Linear Regression Test Score: ", lm.score(x_testsc , y_test))

print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred),2))  
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),2))

Lasso Regression Model

In [ ]:
lasso_001 = Lasso(alpha = 0.5 , max_iter=10e5)

lasso_001.fit(x_trainsc , y_train)


y_pred_lasso = lasso_001.predict(x_testsc)

print("LASSO Regression Train Score: ", lasso_001.score(x_trainsc , y_train))

print("LASSO Regression Test Score: ", lasso_001.score(x_testsc , y_test))


print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred_lasso),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred_lasso),2))  
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso)),2))


coeff_used_002 = np.sum(lasso_001.coef_ != 0)

print("Coefficient used:","\n",coeff_used_002)

In [ ]:
rfe = RFE(lasso_001, n_features_to_select=6)             
rfe = rfe.fit(x_trainsc, y_train)

r2 = sklearn.metrics.r2_score(y_test, y_pred)

scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='r2', cv=5)
scores   

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='r2', cv=folds)
scores  

scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='neg_mean_squared_error', cv=5)
scores

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 10)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'alpha': range(0, 1), 'random_state':range(1, 10,1), 'selection': ['cyclic', 'random'] }]

# step-3: perform grid search
# 3.1 specify model
lasso_001 = Lasso()
lasso_001.fit(x_trainsc , y_train)
rfe = RFE(lasso_001)             

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = lasso_001, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
grid_result= model_cv.fit(x_trainsc, y_train)   
print('Best Score: ', grid_result.best_score_)

print('Best Params: ', grid_result.best_params_)

In [ ]:
lasso_001 = Lasso(alpha = 0 , selection='random', random_state=3)

lasso_001.fit(x_trainsc , y_train)


y_pred_lasso = lasso_001.predict(x_testsc)

print("LASSO Regression Train Score: ", lasso_001.score(x_trainsc , y_train))

print("LASSO Regression Test Score: ", lasso_001.score(x_testsc , y_test))


print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred_lasso),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred_lasso),2))  
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso)),2))


coeff_used_002 = np.sum(lasso_001.coef_ != 0)

print("Coefficient used:","\n",coeff_used_002)

Ridge Regression Model

In [ ]:
rr = Ridge(alpha = 0.5)                                                   # alpha = regularization hyper-parameter

rr.fit(x_trainsc, y_train)


y_pred_rr = rr.predict(x_testsc)

print("Ridge Regression Train Score: ", rr.score(x_trainsc, y_train))
print("Ridge Regression Test Score: ", rr.score(x_testsc, y_test))
print("\n")
print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred_rr),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred_rr),2))
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_rr)),2))



coeff_used_001 = np.sum(rr.coef_ != 0)

print("Coefficient used:","\n",coeff_used_001)

In [ ]:
rfe = RFE(rr, n_features_to_select=6)             
rfe = rfe.fit(x_trainsc, y_train)

r2 = sklearn.metrics.r2_score(y_test, y_pred)

scores = cross_val_score(rr, x_trainsc, y_train, scoring='r2', cv=5)
scores   

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='r2', cv=folds)
scores  

scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='neg_mean_squared_error', cv=5)
scores

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'alpha': list(range(0, 1)),'normalize':[True, False],
                 'random_state':list(range(0, 10)), 'tol': list(range(-100, 100, 50)), 
                 'solver': ['auto', 'svd','cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']}]


# step-3: perform grid search

# 3.1 specify model
rr = Ridge()
rr.fit(x_trainsc, y_train)

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rr,
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds,
                        verbose=1,
                        return_train_score=True)      

# fit the model
grid_result= model_cv.fit(x_trainsc, y_train)   
print('Best Score: ', grid_result.best_score_)

print('Best Params: ', grid_result.best_params_)        

In [ ]:
rr = Ridge(alpha = 0, normalize= True, random_state= 1, solver= 'sag', tol= 50)                                                   # alpha = regularization hyper-parameter

rr.fit(x_trainsc, y_train)


y_pred_rr = rr.predict(x_testsc)

print("Ridge Regression Train Score: ", rr.score(x_trainsc, y_train))
print("Ridge Regression Test Score: ", rr.score(x_testsc, y_test))
print("\n")
print('Mean Absolute Error:', np.round(metrics.mean_absolute_error(y_test, y_pred_rr),2))  
print('Mean Squared Error:', np.round(metrics.mean_squared_error(y_test, y_pred_rr),2))
print('Root Mean Squared Error:', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_rr)),2))



coeff_used_001 = np.sum(rr.coef_ != 0)

print("Coefficient used:","\n",coeff_used_001)

Elastic Net Regression Model

In [ ]:
lm_elastic = ElasticNet(alpha = 0.5)

lm_elastic.fit(x_trainsc, y_train)



y_pred_elastic = lm_elastic.predict(x_testsc)


print("Elastic Net Train Score: ", lm_elastic.score(x_trainsc , y_train))

print("Elastic Net Test Score: ", lm_elastic.score(x_testsc , y_test))


print('Mean Absolute Error: \n', np.round(metrics.mean_absolute_error(y_test, y_pred_elastic),2)) 

print(" ")

print('Mean Squared Error: \n', np.round(metrics.mean_squared_error(y_test, y_pred_elastic),2))

print(" ")

print('Root Mean Squared Error: \n', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_elastic)),2))

coeff_used_003 = np.sum(lm_elastic.coef_ != 0)

print("Coefficient used:","\n",coeff_used_003)

In [ ]:
rfe = RFE(lm_elastic, n_features_to_select=6)             
rfe = rfe.fit(x_trainsc, y_train)

r2 = sklearn.metrics.r2_score(y_test, y_pred)

scores = cross_val_score(rr, x_trainsc, y_train, scoring='r2', cv=5)
scores   

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='r2', cv=folds)
scores  

scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='neg_mean_squared_error', cv=5)
scores

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'alpha': list(range(0, 1)),'fit_intercept': [True, False],
                 'normalize':[True, False],
                 'random_state':list(range(0, 10)), 'tol': list(range(-100, 100, 50)), 
                 'selection': ['cyclic', 'random']}]


# step-3: perform grid search
# 3.1 specify model
lm_elastic = ElasticNet()

lm_elastic.fit(x_trainsc, y_train)
            

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = lm_elastic,
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds,
                        verbose=1,
                        return_train_score=True)
    

# fit the model
grid_result= model_cv.fit(x_trainsc, y_train)   
print('Best Score: ', grid_result.best_score_)

print('Best Params: ', grid_result.best_params_)

In [ ]:
lm_elastic = ElasticNet(alpha= 0, fit_intercept= True, normalize= True, random_state= 0, selection='random', tol= 50)

lm_elastic.fit(x_trainsc, y_train)



y_pred_elastic = lm_elastic.predict(x_testsc)


print("Elastic Net Train Score: ", lm_elastic.score(x_trainsc , y_train))

print("Elastic Net Test Score: ", lm_elastic.score(x_testsc , y_test))


print('Mean Absolute Error: \n', np.round(metrics.mean_absolute_error(y_test, y_pred_elastic),2)) 

print(" ")

print('Mean Squared Error: \n', np.round(metrics.mean_squared_error(y_test, y_pred_elastic),2))

print(" ")

print('Root Mean Squared Error: \n', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_elastic)),2))

coeff_used_003 = np.sum(lm_elastic.coef_ != 0)

print("Coefficient used:","\n",coeff_used_003)

Support Vector Model

In [ ]:
svm = SVR(kernel = 'rbf')
svm.fit(x_trainsc, y_train)



y_pred_svm = svm.predict(x_testsc)


print("Support Vector Model Train Score: ", svm.score(x_trainsc , y_train))

print("Support Vector Model Test Score: ", svm.score(x_testsc , y_test))


print('Mean Absolute Error: \n', np.round(metrics.mean_absolute_error(y_test, y_pred_svm),2)) 

print(" ")

print('Mean Squared Error: \n', np.round(metrics.mean_squared_error(y_test, y_pred_svm),2))

print(" ")

print('Root Mean Squared Error: \n', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_svm)),2))

In [ ]:
rfe = RFE(svm, n_features_to_select=6)             
rfe = rfe.fit(x_trainsc, y_train)

r2 = sklearn.metrics.r2_score(y_test, y_pred)

scores = cross_val_score(rr, x_trainsc, y_train, scoring='r2', cv=5)
scores   

# create a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='r2', cv=folds)
scores  

scores = cross_val_score(lasso_001, x_trainsc, y_train, scoring='neg_mean_squared_error', cv=5)
scores

Decision Tree Regressor

In [ ]:

dTree = DecisionTreeRegressor(random_state=1)
dTree.fit(x_trainsc, y_train)
y_predict = dTree.predict(x_testsc)

print("Decision Tree Model Train Score: ", dTree.score(x_trainsc , y_train))

print("Decision Tree Model Test Score: ", dTree.score(x_testsc , y_test))


print('Mean Absolute Error: \n', np.round(metrics.mean_absolute_error(y_test, y_predict),2)) 

print(" ")

print('Mean Squared Error: \n', np.round(metrics.mean_squared_error(y_test, y_predict),2))

print(" ")

print('Root Mean Squared Error: \n', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_predict)),2))


Random Forest Regressor

In [ ]:
rf_model= RandomForestRegressor(random_state=1)
rf_model.fit(x_trainsc, y_train)
rf_model_predict = rf_model.predict(x_testsc)

print("Random Forest Model Train Score: ", rf_model.score(x_trainsc , y_train))

print("Random Forest Model Test Score: ", rf_model.score(x_testsc , y_test))


print('Mean Absolute Error: \n', np.round(metrics.mean_absolute_error(y_test, rf_model_predict),2)) 

print(" ")

print('Mean Squared Error: \n', np.round(metrics.mean_squared_error(y_test, rf_model_predict),2))

print(" ")

print('Root Mean Squared Error: \n', np.round(np.sqrt(metrics.mean_squared_error(y_test, rf_model_predict)),2))

In [ ]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(dTree)

Conclusion:
Decision tree model performs the best in these conditions